In [3]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import files
uploaded = files.upload()

Saving real_estate_dubai_en-v2 (1).csv to real_estate_dubai_en-v2 (1).csv


In [8]:
# Load dataset
df = pd.read_csv("real_estate_dubai_en-v2 (1).csv")
df.head()

,transaction_id,trans_group_en,procedure_name_en,instance_date,property_type_en,property_sub_type_en,property_usage_en,reg_type_en,area_name_en,building_name_en,...,nearest_metro_en,nearest_mall_en,rooms_en,has_parking,procedure_area,actual_worth,meter_sale_price,no_of_parties_role_1,no_of_parties_role_2,no_of_parties_role_3
0,3-9-2006-163,Gifts,Grant,16-10-2006,Villa,NaN,Residential,Existing Properties,Mankhool,NaN,...,ADCB Metro Station,Dubai Mall,NaN,0,3162.42,12000000.0,3794.56,3.0,1.0,0.0
1,3-9-2019-2944,Gifts,Grant,13-11-2019,Land,NaN,Residential,Existing Properties,Mankhool,NaN,...,ADCB Metro Station,Dubai Mall,NaN,0,209.09,916659.0,4384.04,2.0,4.0,0.0
2,2-13-2001-690,Mortgages,Mortgage Registration,20-08-2001,Building,NaN,Residential / Commercial,Existing Properties,Oud Metha,NaN,...,Oud Metha Metro Station,Dubai Mall,NaN,0,1337.80,4519342.0,3378.19,1.0,1.0,0.0
3,2-13-2020-9477,Mortgages,Mortgage Registration,30-11-2020,Building,NaN,Residential,Existing Properties,Al Bada,NaN,...,Trade Centre Metro Station,Dubai Mall,NaN,0,278.71,2500000.0,8969.90,1.0,1.0,0.0
4,2-13-1999-532,Mortgages,Mortgage Registration,26-04-1999,Villa,NaN,Residential,Existing Properties,Al Bada,NaN,...,Trade Centre Metro Station,Dubai Mall,NaN,0,3626.93,1900000.0,523.86,1.0,1.0,0.0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1548772 entries, 0 to 1548771
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   transaction_id        1548772 non-null  object 
 1   trans_group_en        1548772 non-null  object 
 2   procedure_name_en     1548772 non-null  object 
 3   instance_date         1548772 non-null  object 
 4   property_type_en      1548772 non-null  object 
 5   property_sub_type_en  1227150 non-null  object 
 6   property_usage_en     1548772 non-null  object 
 7   reg_type_en           1548772 non-null  object 
 8   area_name_en          1548772 non-null  object 
 9   building_name_en      1092627 non-null  object 
 10  project_number        1106097 non-null  float64
 11  project_name_en       1106097 non-null  object 
 12  master_project_en     1319601 non-null  object 
 13  nearest_landmark_en   1265574 non-null  object 
 14  nearest_metro_en      1098916 non-

In [10]:
#cheaking missing values
df.isnull().sum()

,0
transaction_id,0
trans_group_en,0
procedure_name_en,0
instance_date,0
property_type_en,0
property_sub_type_en,321622
property_usage_en,0
reg_type_en,0
area_name_en,0
building_name_en,456145


In [11]:
#Cleaning the rooms_en column

#In this step, I focused on the rooms_en column, which contains the number of rooms in each property.

#First, I created a copy of the dataset to keep the original safe.

#Then, I replaced text values like "Studio", "1BR", "2BR" with numbers.

#I ensured the column is numeric by converting it to numbers (pd.to_numeric).

#Missing values were filled using the median number of rooms per property type. This ensures that villas and apartments get appropriate room counts based on their category.

#If all values in a group were missing (like for Land plots), I used 0.

#Finally, I compared the number of missing values before and after cleaning, and displayed the first 20 rows.

In [12]:
# Step 1: Copy dataset for cleaning
# We create a copy of the original dataset (df) to work on.
# This way, the original dataset remains unchanged.
df_clean = df.copy()

# Step 2: Map room labels to numbers
# In the original dataset, the number of rooms is sometimes stored as text
# (e.g., "Studio", "1BR", "2BR"). We replace these with numeric values.
room_mapping = {
    "Studio": 0,
    "1BR": 1,
    "2BR": 2,
    "3BR": 3,
    "4BR": 4,
    "5BR": 5
}
df_clean["rooms_en"] = df_clean["rooms_en"].replace(room_mapping)

# Step 3: Convert to numeric
# Ensure the entire column is numeric. Any values that cannot be converted
# will be turned into NaN (missing values).
df_clean["rooms_en"] = pd.to_numeric(df_clean["rooms_en"], errors="coerce")


# Step 4: Fill missing values with the median per property type

# We define a helper function that fills missing room values using
# the median (middle value) within each property_type_en group.
# Example: Apartments usually have 1–3 rooms, Villas usually have more.

def fill_with_group_median(x):
    median_val = x.median()
    if pd.isna(median_val):
        median_val = 0   # if all values are missing (like Land), use 0
    return x.fillna(round(median_val))

df_clean["rooms_en"] = df_clean.groupby("property_type_en")["rooms_en"].transform(fill_with_group_median)


# Step 5: Compare missing values before and after cleaning

print("Original dataset missing values:", df["rooms_en"].isna().sum())
print("Cleaned dataset missing values:", df_clean["rooms_en"].isna().sum())

# Show first 20 rows for comparison
print(df_clean[["property_type_en", "rooms_en"]].head(20))



Original dataset missing values: 342275
Cleaned dataset missing values: 0
   property_type_en  rooms_en
0             Villa       0.0
1              Land       0.0
2          Building       0.0
3          Building       0.0
4             Villa       0.0
5          Building       0.0
6          Building       0.0
7          Building       0.0
8          Building       0.0
9          Building       0.0
10         Building       0.0
11         Building       0.0
12         Building       0.0
13         Building       0.0
14         Building       0.0
15         Building       0.0
16         Building       0.0
17         Building       0.0
18         Building       0.0
19            Villa       0.0


In [13]:
df_clean.isnull().sum()

,0
transaction_id,0
trans_group_en,0
procedure_name_en,0
instance_date,0
property_type_en,0
property_sub_type_en,321622
property_usage_en,0
reg_type_en,0
area_name_en,0
building_name_en,456145


In [14]:
#Cleaning the property_sub_type_en column

#This column describes the subtype of a property, such as “Flat”, “Townhouse”, or “Penthouse”. However, many rows have missing values.

#To fill these missing values, I used the most common subtype (mode) for each property_type_en group. For example, apartments are most often “Flats”, while villas are often “Townhouses”.

#If an entire group has no data (no subtypes at all), I filled the missing values with "Missing". This ensures that the column is complete and consistent without deleting any rows.

In [15]:
# Step: Clean 'property_sub_type_en'
# The column 'property_sub_type_en' has many missing values.
# Example values: "Penthouse", "Townhouse", "Flat", etc.
# Instead of leaving them empty, we fill them with the most
# frequent value (mode) for each property_type_en group.
#
# Example:
# - For property_type_en = "Apartment", the most common subtype might be "Flat".
# - For property_type_en = "Villa", the most common subtype might be "Townhouse".
#
# If a group has no mode (completely empty), we fill it with "Missing".
df_clean["property_sub_type_en"] = df_clean.groupby("property_type_en")["property_sub_type_en"].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "Missing")
)


In [16]:
df_clean.isnull().sum()

,0
transaction_id,0
trans_group_en,0
procedure_name_en,0
instance_date,0
property_type_en,0
property_sub_type_en,0
property_usage_en,0
reg_type_en,0
area_name_en,0
building_name_en,456145


In [17]:
#Cleaning building, project, and master project names

#These three columns contain location-specific information:

#building_name_en: the exact building name (mainly for apartments).

#project_name_en: the specific project or development (e.g., “Jumeirah Beach Residences”).

#master_project_en: the larger master development or community (e.g., “Dubai Marina”).

#Many rows are missing this information. To fill them, I grouped the data by area_name_en and used the most frequent value (mode) for each area. For example, if most properties in “Dubai Marina” are part of “Marina Tower”, then missing building names in that area will be filled with “Marina Tower”.

#If no mode exists (the area has no data at all), I replaced the missing values with the placeholder "Missing".

#This ensures that all properties have consistent building and project names, while still respecting the geographic area they belong to.

In [18]:

# Step: Clean building, project, and master project names

# These columns ('building_name_en', 'project_name_en', 'master_project_en')
# describe the name of the building, the project, and the master project.
# Many of their values are missing.
#
# To fill them, we group the dataset by 'area_name_en' (the neighborhood).
# Within each area, we replace missing values with the most common value (mode).
#
# Example:
# - In "Dubai Marina", the most common building might be "Marina Tower".
#   If a record in that area is missing its building name, we fill it with "Marina Tower".
# - The same logic applies to project_name_en and master_project_en.
#
# If a group has no mode (completely empty), we fill it with "Missing".
for col in ["building_name_en", "project_name_en", "master_project_en"]:
    df_clean[col] = df_clean.groupby("area_name_en")[col].transform(
        lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "Missing")
    )



In [19]:
df_clean.isnull().sum()

,0
transaction_id,0
trans_group_en,0
procedure_name_en,0
instance_date,0
property_type_en,0
property_sub_type_en,0
property_usage_en,0
reg_type_en,0
area_name_en,0
building_name_en,0


In [20]:
#Cleaning nearest metro and nearest mall

#These columns describe the nearest transport and commercial facilities for each property:

#nearest_metro_en: the closest metro station.

#nearest_mall_en: the closest shopping mall.

#Since many values were missing, I grouped the data by area_name_en and filled missing values with the most frequent facility (mode) for that area.

#For example, in “Dubai Marina”, most properties are close to “DAMAC Metro Station” and “Dubai Marina Mall”. If a property in that area was missing these values, I filled them with the most common ones.

#If no mode exists for an area, I filled the missing values with "None". This makes sure that every property has a value, but we also clearly distinguish areas with no available data.

In [21]:

# Step: Clean nearest metro and nearest mall

# These two columns show the nearest metro station and the nearest shopping mall
# to each property. Many rows are missing this information.
#
# To fill them, we group by 'area_name_en' (the district).
# Within each area, missing values are replaced with the most common value (mode).
#
# Example:
# - In "Dubai Marina", most properties are close to "DAMAC Metro Station".
#   If a property in this area has a missing metro, we fill it with "DAMAC Metro Station".
# - Similarly, for malls in the same area, if most properties list "Dubai Marina Mall",
#   missing values will be filled with "Dubai Marina Mall".
#
# If no mode is available for an area (no data at all), we fill the missing values with "None".
for col in ["nearest_metro_en", "nearest_mall_en"]:
    df_clean[col] = df_clean.groupby("area_name_en")[col].transform(
        lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "None")
    )


In [22]:
df_clean.isnull().sum()

,0
transaction_id,0
trans_group_en,0
procedure_name_en,0
instance_date,0
property_type_en,0
property_sub_type_en,0
property_usage_en,0
reg_type_en,0
area_name_en,0
building_name_en,0


In [23]:
#Cleaning the nearest_landmark_en column

#This column indicates the nearest landmark for a property, such as “Burj Khalifa” or “Palm Jumeirah”. Many values were missing.

#To handle this, I grouped the dataset by area_name_en and filled missing values with the most common landmark (mode) in that area. For example, in “Downtown Dubai”, missing landmarks are most often replaced with “Burj Khalifa”.

#If an area has no data at all, the missing value is filled with "None". This ensures that the column is complete while still distinguishing between properties with real landmarks and those without data.#

In [24]:

# Step: Clean nearest landmark

# The column 'nearest_landmark_en' shows the closest well-known landmark
# (e.g., Burj Khalifa, Palm Jumeirah) for each property.
# Many rows are missing values in this column.
#
# To fill them, we group the data by 'area_name_en'.
# Within each area, we replace missing values with the most frequent landmark (mode).
#
# Example:
# - In "Downtown Dubai", most properties list "Burj Khalifa" as the nearest landmark.
#   If a property in this area is missing its landmark, we fill it with "Burj Khalifa".
#
# If no mode exists for an area (completely empty), we fill missing values with "None".
df_clean["nearest_landmark_en"] = df_clean.groupby("area_name_en")["nearest_landmark_en"].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "None")
)


In [25]:
#Dropping the project_number column

#The column project_number is just a numerical identifier for projects. It does not contain meaningful information for analysis because the project is already described in detail by project_name_en and master_project_en.

#ID columns like this are not useful for modeling and can even cause problems (e.g., by acting as artificial unique identifiers). For this reason, I dropped the column to simplify the dataset and avoid redundancy.

In [26]:

# Step: Drop the 'project_number' column

# The column 'project_number' is simply an internal ID number for each project.
# It does not carry useful information for analysis or prediction,
# because the same project already appears in 'project_name_en'.
#
# Keeping ID columns can create noise or data leakage in machine learning models.
# Therefore, we safely drop this column.
df_clean = df_clean.drop(columns=["project_number"])



In [27]:
df_clean.isnull().sum()

,0
transaction_id,0
trans_group_en,0
procedure_name_en,0
instance_date,0
property_type_en,0
property_sub_type_en,0
property_usage_en,0
reg_type_en,0
area_name_en,0
building_name_en,0


In [28]:
#Cleaning the party role columns

#The columns no_of_parties_role_1, no_of_parties_role_2, and no_of_parties_role_3 represent the number of parties in different roles in each transaction (buyers, sellers, brokers). Almost all values are complete, with only 968 missing rows out of more than 1.5 million.

#I filled the missing values with the median. The reason is that most transactions involve 1 buyer and 1 seller, so the median value is stable and not influenced by rare outlier cases (like a group of 10 buyers). This choice keeps the dataset realistic while ensuring that no missing values remain in these important columns.

In [29]:

# Step: Fill missing values in party role columns

# These three columns represent the number of parties involved in the transaction:
# - no_of_parties_role_1 → usually the number of buyers
# - no_of_parties_role_2 → usually the number of sellers
# - no_of_parties_role_3 → sometimes brokers or intermediaries
#
# Almost all rows already have values, but about 968 rows out of 1.5 million are missing.
# Since most transactions involve 1 buyer and 1 seller, the median value is usually 1.
#
# By filling missing values with the median, we preserve the natural distribution of the data
# without letting rare outliers (like group transactions with 10 buyers) influence the result.
for col in ["no_of_parties_role_1", "no_of_parties_role_2", "no_of_parties_role_3"]:
    df_clean[col] = df_clean[col].fillna(df_clean[col].median())



In [30]:
df_clean.isnull().sum()

,0
transaction_id,0
trans_group_en,0
procedure_name_en,0
instance_date,0
property_type_en,0
property_sub_type_en,0
property_usage_en,0
reg_type_en,0
area_name_en,0
building_name_en,0


In [34]:
#Cleaning and transforming the instance_date column

#The instance_date column stores the date of the transaction, but it was in text format. I converted it into a proper datetime object so that it can be used in time-based analysis.

#From this cleaned date, I extracted three new features:

#Year: to study long-term price and volume trends over multiple years.

#Month: to capture seasonal patterns in real estate activity.

#Day of week: to analyze whether transactions are more frequent on weekdays or weekends.

#This transformation makes the dataset much more powerful for exploratory analysis and predictive modeling.

#The dataset stores transaction dates in day-month-year format (e.g., 15-03-2021). Pandas by default expects month-day-year, which caused a parsing warning.
#To resolve this, I explicitly set dayfirst=True in pd.to_datetime(). This ensures that dates like 15-03-2021 are correctly parsed as 15 March 2021 instead of March 15, 2021.

In [35]:


# Step: Convert 'instance_date' with correct day-first format

# The dates in this dataset are in format: day-month-year (e.g., 15-03-2021).
# By default, Pandas assumes month-day-year, which causes warnings and mistakes.
#
# To fix this, we set dayfirst=True when parsing dates.
df_clean["instance_date"] = pd.to_datetime(
    df_clean["instance_date"],
    errors="coerce",
    dayfirst=True   # important for Dubai dataset (DD-MM-YYYY format)
)

# Extract time-based features
df_clean["year"] = df_clean["instance_date"].dt.year
df_clean["month"] = df_clean["instance_date"].dt.month
df_clean["day_of_week"] = df_clean["instance_date"].dt.dayofweek



In [39]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1535582 entries, 0 to 1548771
Data columns (total 26 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   transaction_id        1535582 non-null  object        
 1   trans_group_en        1535582 non-null  object        
 2   procedure_name_en     1535582 non-null  object        
 3   instance_date         1535578 non-null  datetime64[ns]
 4   property_type_en      1535582 non-null  object        
 5   property_sub_type_en  1535582 non-null  object        
 6   property_usage_en     1535582 non-null  object        
 7   reg_type_en           1535582 non-null  object        
 8   area_name_en          1535582 non-null  object        
 9   building_name_en      1535582 non-null  object        
 10  project_name_en       1535582 non-null  object        
 11  master_project_en     1535582 non-null  object        
 12  nearest_landmark_en   1535582 non-null  object 

In [36]:
#Final Outlier Filtering

#In this step, I applied all outlier filters together:

#actual_worth: kept between 50,000 AED and 1 billion AED.

#rooms_en: kept between 0–15 rooms.

#procedure_area: kept between 10–20,000 m².

#meter_sale_price: kept between 100–100,000 AED per m².

#no_of_parties_role_1, 2, 3: capped at 10 parties maximum.

#This ensures the dataset only includes realistic and valid transactions, removing extreme values that could bias analysis or machine learning models.

In [37]:

# Step: Outlier removal for multiple features


# Apply all filters step by step in one block

df_clean = df_clean[
    (df_clean["actual_worth"] >= 50_000) & (df_clean["actual_worth"] <= 1_000_000_000) &  # realistic transaction prices
    (df_clean["rooms_en"] >= 0) & (df_clean["rooms_en"] <= 15) &                          # realistic number of rooms
    (df_clean["procedure_area"] > 10) & (df_clean["procedure_area"] < 20_000) &           # property area range
    (df_clean["meter_sale_price"] > 100) & (df_clean["meter_sale_price"] < 100_000) &    # price per m² range
    (df_clean["no_of_parties_role_1"] <= 10) &                                           # limit buyers
    (df_clean["no_of_parties_role_2"] <= 10) &                                           # limit sellers
    (df_clean["no_of_parties_role_3"] <= 10)                                             # limit brokers
]

# Show dataset shape after removing outliers
print("Shape after outlier filtering:", df_clean.shape)


Shape after outlier filtering: (1535582, 26)


In [47]:
#Final check


# Check if there are still any missing values
print("Total missing values remaining:", df_clean.isnull().sum().sum())

# Check final dataset size
print("Final shape of dataset:", df_clean.shape)




Total missing values remaining: 0
Final shape of dataset: (1535578, 26)


In [41]:
df_clean.isnull().sum().sort_values(ascending=False).head(10)


,0
instance_date,4
day_of_week,4
year,4
month,4
trans_group_en,0
property_sub_type_en,0
property_usage_en,0
procedure_name_en,0
property_type_en,0
transaction_id,0


In [48]:
#Dropping rows with missing dates

#When I converted instance_date to datetime, 4 rows had invalid or missing values. These became NaT (Not a Time). As a result, their derived features (year, month, day_of_week) were also missing.

#Since these are only 4 rows out of more than 1.5 million records, removing them does not affect the dataset quality. By dropping them, I ensure that the dataset is now 100% free of missing values and ready for analysis or modeling.

In [43]:
df_clean = df_clean.dropna(subset=["instance_date"])


In [50]:
#Final check and export after dropping invalid dates

# Double-check for missing values (should be zero now)
print("Total missing values remaining:", df_clean.isnull().sum().sum())

# Check final dataset size
print("Final shape of dataset:", df_clean.shape)

# Save cleaned dataset as CSV
df_clean.to_csv("dubai_real_estate_clean.csv", index=False)

print("Final cleaned dataset saved as 'dubai_real_estate_clean.csv'")


Total missing values remaining: 0
Final shape of dataset: (1535578, 26)
✅ Final cleaned dataset saved as 'dubai_real_estate_clean.csv'


In [51]:
from google.colab import files
files.download("dubai_real_estate_clean.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>